# Business understanding

The objective is to perform a time series analysis and explore how well Airbnb performed for the past two years (2019 - 2020), taking COVID-19 effect always in consideration.

Hong Kong is a specific city of interst for this analysis.

his notebook only serves to proprocess extremely large datasets, and the actual analysis will be done in a seperate notebook `analysis.ipynb`

Following ETL pipeline : Extract, Transform and Load

In [1]:
import pandas as pd
import numpy as np

import os
import re
import random
import gzip

In [60]:
from sqlalchemy import create_engine

In [2]:
%config Completer.use_jedi = False

# STEP1 | Extract

Datasets were downloaded from [Inside Airbnb](http://insideairbnb.com/get-the-data.html), and are saved as compressed gzip file. 

## Explore data

### Explore file directory
There are 20 data files categorized by published date (year, month). All are compressed by gzip. It is important to note that some periods are missing between 2020-06 and 2020-09.

In [61]:
DATA_DIR = sorted(os.listdir('data'))
print(len(DATA_DIR), '\n', DATA_DIR)

20 
 ['calendar_201901.csv.gz', 'calendar_201902.csv.gz', 'calendar_201903.csv.gz', 'calendar_201904.csv.gz', 'calendar_201905.csv.gz', 'calendar_201906.csv.gz', 'calendar_201907.csv.gz', 'calendar_201908.csv.gz', 'calendar_201909.csv.gz', 'calendar_201910.csv.gz', 'calendar_201911.csv.gz', 'calendar_201912.csv.gz', 'calendar_202001.csv.gz', 'calendar_202002.csv.gz', 'calendar_202003.csv.gz', 'calendar_202004.csv.gz', 'calendar_202005.csv.gz', 'calendar_202006.csv.gz', 'calendar_202010.csv.gz', 'calendar_202011.csv.gz']


## Load data 
Use `data/calendar_201901.csv.gz` for testing. Opening gzip file with pandas library.

In [62]:
df_test = pd.read_csv('data/calendar_201901.csv.gz', compression='gzip') 

In [63]:
df_test.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,944283,2019-01-16,f,$941.00,$941.00,5,125
1,554138,2019-01-16,f,$750.00,$750.00,2,1125
2,554138,2019-01-17,f,$750.00,$750.00,2,1125
3,554138,2019-01-18,f,$750.00,$750.00,2,1125
4,554138,2019-01-19,f,$750.00,$750.00,2,1125


In [64]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3824105 entries, 0 to 3824104
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   listing_id      int64 
 1   date            object
 2   available       object
 3   price           object
 4   adjusted_price  object
 5   minimum_nights  int64 
 6   maximum_nights  int64 
dtypes: int64(3), object(4)
memory usage: 204.2+ MB


---
# STEP2 | TRANFORM

## Remove duplicates

## Remove columns 
For this analysis, minimum_nights and maxmium_nights are not necessary.
Also, price and adjusted_price share duplicated information. price column is to be dropped and adjusted_price column to be renamed as price. 

In [66]:
# Drop unnessary columns
cols_drop = ['price', 'minimum_nights', 'maximum_nights']
df_test.drop(columns = cols_drop, inplace=True)

In [67]:
# Rename adjusted_price column to price
df_test = df_test.rename(columns={'adjusted_price': 'price'})

In [68]:
df_test.head()

,listing_id,date,available,price
0,944283,2019-01-16,f,$941.00
1,554138,2019-01-16,f,$750.00
2,554138,2019-01-17,f,$750.00
3,554138,2019-01-18,f,$750.00
4,554138,2019-01-19,f,$750.00


## Drop null

In [70]:
# Check for null values
df_test.isnull().sum() # none!

listing_id    0
date          0
available     0
price         0
dtype: int64

## Drop duplicates

In [73]:
df_test[df_test.duplicated()] # no duplicates!

,listing_id,date,available,price


## Converting data 
Converting data to faciliate future analysis i.e. aggregation, groupby, etc

### Convert datetime object to datetime index

In [74]:
# Convert date string to datetime object
df_test['date'] = pd.to_datetime(df_test['date'])

In [75]:
# Set date index
df_test = df_test.set_index('date')

In [76]:
df_test.head()

,listing_id,available,price
date,,,
2019-01-16,944283,f,$941.00
2019-01-16,554138,f,$750.00
2019-01-17,554138,f,$750.00
2019-01-18,554138,f,$750.00
2019-01-19,554138,f,$750.00


### Convert `available` column to binary numbers

In [77]:
df_test['available'] = df_test['available'].map({'t': 1, 'f':0})

### Convert `price` object to float

In [78]:
# Firstly we need to remove sign $ and ,
seperators = ['$', ',']

for sep in seperators : 
    df_test['price'] = df_test['price'].str.replace(sep, '') 

In [79]:
# Convert date type to float
df_test['price'] = df_test['price'].astype(float)

In [80]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3824105 entries, 2019-01-16 to 2020-01-15
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   listing_id  int64  
 1   available   int64  
 2   price       float64
dtypes: float64(1), int64(2)
memory usage: 116.7 MB


## Filter data

The dataset contains period beyond the actual month which is just projected based on the current month data. Any periods beyond the specific month can not provide market-based performance information, so need to be dropped. For example 2019-05 dataset is projected until 2020-05.

Furthermore, each dataset does not have a clear collection period provided. For example, 2019-05 data starts from 2019-05-14, whereas 2019-01 data starts from 2019-01-16.  

### Filtering method
As it is hard to filter data exactly for the actual month, I will : 

1. group the data by each date. It is fine as I will not deal with individual 'listing' level analysis, but rather with daily time series analysis with aggrated data (by average). 

2. save the dataframe as `df_analysis`

3. remove 'listing_id' column as it will not give any information after aggregation 

4. only collect the data up to +35 days from the available start date. This is to try to avoid missing data when concatenating different files, and at the same time to limit the data duplication as little as possible (less duplicated then the original +365 days for sure).

In [81]:
# Group by date
df_analysis = df_test.groupby('date').mean().copy()

# Remove listing_id column
df_analysis = df_analysis.drop(columns='listing_id')

In [82]:
df_analysis.head()

,available,price
date,,
2019-01-16,0.317744,813.774172
2019-01-17,0.198721,817.817123
2019-01-18,0.255894,866.878591
2019-01-19,0.235086,882.824854
2019-01-20,0.309249,830.250549


In [83]:
# Limit the period to +35 days from the available start date
df_analysis = df_analysis.iloc[:35]
df_analysis.tail()

,available,price
date,,
2019-02-15,0.495180,903.958862
2019-02-16,0.473513,915.112532
2019-02-17,0.507015,857.166937
2019-02-18,0.520092,849.563234
2019-02-19,0.523146,849.154434


<a id="wrangle"></a>
## Create a function for wrangling

In [84]:
def wrangle_data(df) :
    '''
    Wrangle and transform a dataframe into a new dataframe ready for analysis.
    Checking for null and duplicates are both important operations to ensure data quality.
    Therefore, it will seperately be done after wrangling with this funciton.
    
    INPUT  : unclean dataframe
    OUTPUT : a new dataframe ready for analysis     
    '''
    
    # Copy dataframe for cleaning
    df_copy = df.copy() 
    
    # Remove unnecssary columns and rename adjusted_price column to price
    cols_drop = ['price', 'minimum_nights', 'maximum_nights']
    df_copy.drop(columns=cols_drop,inplace=True)
    
    # Rename adjusted_price column to price
    df_copy = df_copy.rename(columns={'adjusted_price': 'price'})
    
    # Convert datetime object to datetime index
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    df_copy = df_copy.set_index('date')
    
    # Convert 'avaiable' column to binary numbers
    df_copy['available'] = df_copy['available'].map({'t':1, 'f':0})
    
    # Convert 'price' column to float date type
    seperators = ['$', ',']
    for sep in seperators : 
        df_copy['price'] = df_copy['price'].str.replace(sep, '')
    
    df_copy['price'] = df_copy['price'].astype(float)
        
    # Create analysis dataframe - aggregate records by date
    df_analysis = df_copy.groupby('date').mean().copy()
    
    # Remove listing_id column (no longer useful)
    df_analysis = df_analysis.drop(columns='listing_id')
    
    # Limit the period to +35 days from the available start date
    df_analysis = df_analysis.iloc[:35].reset_index()
    
    return df_analysis

In [85]:
# Testing 'wrangle_data' function with a random data
rand_idx = np.random.randint(len(DATA_DIR))
rand_file = os.path.join('data', DATA_DIR[rand_idx])

# Load a new gzip data for testing
df_rand = pd.read_csv(rand_file, compression='gzip')

In [86]:
df_rand.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,1907254,2020-11-10,t,"$1,124.00","$1,124.00",90,365
1,1907254,2020-11-11,f,"$1,124.00","$1,124.00",90,365
2,1907254,2020-11-12,f,"$1,124.00","$1,124.00",90,365
3,1907254,2020-11-13,f,"$1,124.00","$1,124.00",90,365
4,1907254,2020-11-14,f,"$1,124.00","$1,124.00",90,365


In [87]:
df_rand_clean = wrangle_data(df_rand)
df_rand_clean.head()

,date,available,price
0,2020-11-10,0.730003,703.635483
1,2020-11-11,0.441738,704.146139
2,2020-11-12,0.676723,704.775810
3,2020-11-13,0.698312,730.108220
4,2020-11-14,0.719208,736.865486


In [88]:
df_rand_clean.tail()

,date,available,price
30,2020-12-10,0.830888,710.936064
31,2020-12-11,0.827982,735.334763
32,2020-12-12,0.827429,738.651813
33,2020-12-13,0.830197,712.287296
34,2020-12-14,0.829643,711.867977


---
# `STEP3` | Load date
Use sqlalchemy engine and load data to a database.
Set the name of database to be `airbnb_tsa.db` and the table name to be `performance` (analysis for Airbnb performance).

Repeating the previous `import` and `transform`, on a new dataset `data/calendar_201901.csv.gz` 

In [107]:
# Import
df = pd.read_csv('data/calendar_201901.csv.gz', compression='gzip')
df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,944283,2019-01-16,f,$941.00,$941.00,5,125
1,554138,2019-01-16,f,$750.00,$750.00,2,1125
2,554138,2019-01-17,f,$750.00,$750.00,2,1125
3,554138,2019-01-18,f,$750.00,$750.00,2,1125
4,554138,2019-01-19,f,$750.00,$750.00,2,1125


In [108]:
# Transform
analysis = wrangle_data(df)
analysis.head()

,date,available,price
0,2019-01-16,0.317744,813.774172
1,2019-01-17,0.198721,817.817123
2,2019-01-18,0.255894,866.878591
3,2019-01-19,0.235086,882.824854
4,2019-01-20,0.309249,830.250549


In [109]:
# Create an engine for loading the data
engine = create_engine('sqlite:///airbnb_tsa.db')

In [110]:
# Save it to 'performance' table
# To stack data from multiple files, set if_exists='append'
analysis.to_sql('performance', engine, if_exists='replace', index=False)

In [112]:
# Check if data is storted well 
engine.execute("SELECT * FROM performance").fetchall()[:10]

[('2019-01-16 00:00:00.000000', 0.31774362890140306, 813.7741719958003),
 ('2019-01-17 00:00:00.000000', 0.1987210079221151, 817.8171232222965),
 ('2019-01-18 00:00:00.000000', 0.25589386274696957, 866.8785911997709),
 ('2019-01-19 00:00:00.000000', 0.23508637968884222, 882.8248544430658),
 ('2019-01-20 00:00:00.000000', 0.3092488307721676, 830.2505488212274),
 ('2019-01-21 00:00:00.000000', 0.33206070440011454, 833.8614107091724),
 ('2019-01-22 00:00:00.000000', 0.34303712894912664, 832.8711463205116),
 ('2019-01-23 00:00:00.000000', 0.3550634723680443, 833.7834303712895),
 ('2019-01-24 00:00:00.000000', 0.37329388183640355, 832.9353822659158),
 ('2019-01-25 00:00:00.000000', 0.36355827049727973, 884.2108427985111)]

## Create data pipeline 
For the entire data files

In [131]:
def extract_data(file_path) :
    df = pd.read_csv(file_path, compression='gzip')    
    return df
    
# def wrangle_data(df) 

def store_data(df, table='performance', db='airbnb_tsa.db') : 
    # Create engine
    engine = create_engine(f"sqlite:///{db}") # be aware to wrap in ""(double quotes)
    df.to_sql(table, engine, if_exists='append', index=False)

In [133]:
def main() : 
    
    # Path to data files     
    DIR_NAME = 'data'
    DATA_FILES = os.listdir('data')
    
    for file in sorted(DATA_FILES) : 
        
        # Instantiate gzfile to open
        gzfile = ''
        
        # Check if file is gzip
        if not file.endswith('.gz') : 
            continue

        else : 
            gzfile = os.path.join(DIR_NAME, file)
                
        # Extract gzfile to a dataframe
        df = extract_data(gzfile)
        
        # Clean dataframe 
        df = wrangle_data(df)
        
        # Store clean data to 'airbnb_tsa.db' database
        store_data(df)
        
        print(f'Successfully stored {file} to db')
        
main()

Successfully stored calendar_201901.csv.gz to db
Successfully stored calendar_201902.csv.gz to db
Successfully stored calendar_201903.csv.gz to db
Successfully stored calendar_201904.csv.gz to db
Successfully stored calendar_201905.csv.gz to db
Successfully stored calendar_201906.csv.gz to db
Successfully stored calendar_201907.csv.gz to db
Successfully stored calendar_201908.csv.gz to db
Successfully stored calendar_201909.csv.gz to db
Successfully stored calendar_201910.csv.gz to db
Successfully stored calendar_201911.csv.gz to db
Successfully stored calendar_201912.csv.gz to db
Successfully stored calendar_202001.csv.gz to db
Successfully stored calendar_202002.csv.gz to db
Successfully stored calendar_202003.csv.gz to db
Successfully stored calendar_202004.csv.gz to db
Successfully stored calendar_202005.csv.gz to db
Successfully stored calendar_202006.csv.gz to db
Successfully stored calendar_202010.csv.gz to db
Successfully stored calendar_202011.csv.gz to db


In [134]:
# Check the data with a query
engine.execute('SELECT * FROM performance').fetchall()[:50]

[('2019-01-16 00:00:00.000000', 0.31774362890140306, 813.7741719958003),
 ('2019-01-17 00:00:00.000000', 0.1987210079221151, 817.8171232222965),
 ('2019-01-18 00:00:00.000000', 0.25589386274696957, 866.8785911997709),
 ('2019-01-19 00:00:00.000000', 0.23508637968884222, 882.8248544430658),
 ('2019-01-20 00:00:00.000000', 0.3092488307721676, 830.2505488212274),
 ('2019-01-21 00:00:00.000000', 0.33206070440011454, 833.8614107091724),
 ('2019-01-22 00:00:00.000000', 0.34303712894912664, 832.8711463205116),
 ('2019-01-23 00:00:00.000000', 0.3550634723680443, 833.7834303712895),
 ('2019-01-24 00:00:00.000000', 0.37329388183640355, 832.9353822659158),
 ('2019-01-25 00:00:00.000000', 0.36355827049727973, 884.2108427985111),
 ('2019-01-26 00:00:00.000000', 0.34943208933855113, 897.2749832967453),
 ('2019-01-27 00:00:00.000000', 0.39515128376443637, 835.9721294263625),
 ('2019-01-28 00:00:00.000000', 0.41767681588240907, 831.9420635678152),
 ('2019-01-29 00:00:00.000000', 0.4399160064904076, 83

In [135]:
engine.execute('SELECT * FROM performance').fetchall()[-50:]

[('2020-11-14 00:00:00.000000', 0.8095436260230268, 733.692606464142),
 ('2020-11-15 00:00:00.000000', 0.8181439866833126, 706.6265778887501),
 ('2020-11-16 00:00:00.000000', 0.8218893050353725, 704.7856845609655),
 ('2020-11-17 00:00:00.000000', 0.8228603134970176, 704.6896934387571),
 ('2020-11-18 00:00:00.000000', 0.8203634345956443, 704.9101123595506),
 ('2020-11-19 00:00:00.000000', 0.8207795810792066, 711.9139963933972),
 ('2020-11-20 00:00:00.000000', 0.8211957275627687, 729.8650298238314),
 ('2020-11-21 00:00:00.000000', 0.8198085726175613, 725.7084200305175),
 ('2020-11-22 00:00:00.000000', 0.8272992093216812, 699.8343736995422),
 ('2020-11-23 00:00:00.000000', 0.8293799417394923, 698.1195727562769),
 ('2020-11-24 00:00:00.000000', 0.8306283811901789, 697.8042724372312),
 ('2020-11-25 00:00:00.000000', 0.8292412262449715, 698.3505340546538),
 ('2020-11-26 00:00:00.000000', 0.8306283811901789, 698.6029962546817),
 ('2020-11-27 00:00:00.000000', 0.8303509502011375, 723.303370786

In [ ]:
df.drop_duplicates(keep: 'last')

It looks okay! Still there are some things to remember when loading to use the data for analysis.
    
1. Check fo null values    
2. Check for duplicates
3. Check for missing dates (some periods are missing, which may need imputation)

When it comes to duplicates, we filtered +35 days records from the start date in order to avoid any missing date (as much as possible) from concatenation. As a result, there are some duplicated dates seen in the database (i.e. '2020-11-14', etc), which need to be dropped. When using pandas\'s `drop_duplicates` method, parameter must be set to `keep='last'` in order to keep more recent data.
